In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review


In [ ]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Compiled_data.csv')

In [ ]:
df.head()

Reviews  Rating
0  \n\n  Great device and working good. Nice cove...       5
1  \n\n  First of all its little bit tough to ins...       5
2  \n\n  Good packaging and timely delivered. Abo...       5
3                                  \n\n  So good\n\n       5
4  \n\n  It's good productI am using it as a home...       5

In [ ]:
df['Reviews']=[i.split('\n\n')[1] for i in df['Reviews']]


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
from wordcloud import WordCloud

In [ ]:
ps=PorterStemmer()
wordnet=WordNetLemmatizer()

In [ ]:
corpus=[]
for i in range(len(df)):
    r=re.sub('[^a-zA-Z0-9]',' ',df['Reviews'][i])
    r=r.lower()
    r=r.split()
    r=[wordnet.lemmatize(k) for k in r]# if k not in ll]
    r=' '.join(r)
    corpus.append(r)

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
nltk.download('stopwords')
lu=set(stopwords.words('english'))
ll=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
    'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 
    'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 
    'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 
    'only', 'own', 'same', 'so', 'than' 's', 't', 'can', 'will', 'just', 
    'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain',
    'ma']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['Reviews'] = df['Reviews'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
df['Reviews'] = df['Reviews'].apply(lambda x: re.sub('[^a-zA-Z0-9]',' ',x))
df['Reviews'] = df['Reviews'].apply(lambda x: ' '.join(x for x in x.split() if x not in ll))

df['Reviews']

0        great device working good nice coverage throug...
1        first little bit tough install afterwards work...
2        good packaging timely delivered printer truly ...
3                                                     good
4        s good producti using home theatrevalue moneyb...
                               ...                        
24870    no drivers mac canon claims cannot find no web...
24871    didn get bill products product start troubling...
24872    photo copy not workingwithout contact phon not...
24873    battery back very poor 5days ordered replaceme...
24874    wi fi made very cheap quality not worth price ...
Name: Reviews, Length: 24875, dtype: object

In [ ]:
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(df['Reviews'].values)
X = tokenizer.texts_to_sequences(df['Reviews'].values)
X = pad_sequences(X)

In [ ]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, recurrent_dropout=0.2))
#model.add(Dense(25,activation='relu'))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 329, 120)          60000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 176)               209088    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 885       
Total params: 269,973
Trainable params: 269,973
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
y=pd.get_dummies(df['Rating'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25,random_state=99)

In [ ]:
batch_size=128
model.fit(X_train, y_train, epochs = 2, batch_size=batch_size)

Epoch 1/2
146/146 [==============================] - 204s 1s/step - loss: 1.4967 - accuracy: 0.3086
Epoch 2/2
146/146 [==============================] - 198s 1s/step - loss: 1.2314 - accuracy: 0.4527


In [ ]:
model.evaluate(X_test,y_test)

195/195 [==============================] - 27s 131ms/step - loss: 1.1613 - accuracy: 0.4835


[1.1613175868988037, 0.48351824283599854]